## Idea:
* Use month trend from [Only Trends](https://www.kaggle.com/code/vitalykudelya/only-trends)
* Divide NPX values of a protein into several groups and find the best shift after month trend predicitons for each group
* Sum predictions from the month trend and the corresponding NPX group shift

# ライブラリ,設定

In [1]:
from collections import defaultdict
from tqdm.auto import tqdm
import plotly.express as px
import amp_pd_peptide
from scipy.optimize import minimize
import pandas as pd
import numpy as np
import warnings
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold, GroupKFold
warnings.simplefilter('ignore')
from sklearn.linear_model import LinearRegression as LR
import optuna.integration.lightgbm as lgbm_opt
from matplotlib.ticker import MaxNLocator
import lightgbm as lgbm
import random
from scipy.optimize import minimize
import category_encoders as ce
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.metrics import confusion_matrix
import pickle
import sys
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)

In [2]:
sys.path.append('/kaggle/input')
from lgbm20230515v2refuct import predict_lgbm_v2_refuct as predict_lgbm

In [3]:
model_path = '/kaggle/input/lgbmmodels/lgbm_models'

In [4]:
class CFG:
    debug = False
    num_boost = 10000
    num_early = 30
    target_enc = False
    learn_optuna = True
    learn_local = False

In [5]:
def smape_plus_1(y_true, y_pred):
    y_true_plus_1 = y_true + 1
    y_pred_plus_1 = y_pred + 1
    metric = np.zeros(len(y_true_plus_1))
    
    numerator = np.abs(y_true_plus_1 - y_pred_plus_1)
    denominator = ((np.abs(y_true_plus_1) + np.abs(y_pred_plus_1)) / 2)
    
    mask_not_zeros = (y_true_plus_1 != 0) | (y_pred_plus_1 != 0)
    metric[mask_not_zeros] = numerator[mask_not_zeros] / denominator[mask_not_zeros]
    
    return 100 * np.nanmean(metric)

## Generate Train Dataset

In [6]:
train_clinical_all = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')
proteins = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv')
proteins_features = pd.pivot_table(proteins, values='NPX', index='visit_id', columns='UniProt', aggfunc='sum')

train_clinical_all = train_clinical_all.merge(
    proteins_features,
    left_on='visit_id',
    right_index=True,
    how='left'
)

In [7]:
train_clinical_all[proteins_features.columns] = train_clinical_all.groupby(
    'patient_id'
)[proteins_features.columns].fillna(method='ffill')

In [8]:
train_clinical_all['pred_month'] = train_clinical_all['visit_month']

for plus_month in [6, 12, 24]:
    train_shift = train_clinical_all[['patient_id', 'visit_month', 'pred_month', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']].copy()
    train_shift['visit_month'] -= plus_month
    train_shift.rename(columns={f'updrs_{i}': f'updrs_{i}_plus_{plus_month}' for i in range(1, 5)}, inplace=True)
    train_shift.rename(columns={'pred_month': f'pred_month_plus_{plus_month}'}, inplace=True)
    train_clinical_all = train_clinical_all.merge(train_shift, how='left', on=['patient_id', 'visit_month'])

train_clinical_all.rename(columns={f'updrs_{i}': f'updrs_{i}_plus_0' for i in range(1, 5)}, inplace=True)
train_clinical_all.rename(columns={'pred_month': f'pred_month_plus_0'}, inplace=True)
train_clinical_all

,visit_id,patient_id,visit_month,updrs_1_plus_0,updrs_2_plus_0,updrs_3_plus_0,updrs_4_plus_0,upd23b_clinical_state_on_medication,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,O60888,O75144,O75326,O94919,P00441,P00450,P00734,P00736,P00738,P00746,P00747,P00748,P00751,P01008,P01009,P01011,P01019,P01023,P01024,P01031,P01033,P01034,P01042,P01344,P01591,P01594,P01608,P01621,P01717,P01780,P01833,P01834,P01857,P01859,P01860,P01861,P01876,P01877,P02452,P02647,P02649,P02652,P02655,P02656,P02671,P02675,P02679,P02747,P02748,P02749,P02750,P02751,P02753,P02760,P02763,P02765,P02766,P02768,P02774,P02787,P02790,P02792,P04004,P04075,P04156,P04180,P04196,P04207,P04211,P04216,P04217,P04275,P04406,P04433,P05060,P05067,P05090,P05155,P05156,P05408,P05452,P05546,P06310,P06396,P06454,P06681,P06727,P07195,P07225,P07333,P07339,P07602,P07711,P07858,P07998,P08123,P08133,P08253,P08294,P08493,P08571,P08603,P08637,P08697,P09104,P09486,P09871,P10451,P10643,P10645,P10909,P11142,P11277,P12109,P13473,P13521,P13591,P13611,P13671,P13987,P14174,P14314,P14618,P16035,P16070,P16152,P16870,P17174,P17936,P18065,P19021,P19652,P19823,P19827,P20774,P20933,P23083,P23142,P24592,P25311,P27169,P30086,P31997,P32754,P35542,P36222,P36955,P36980,P39060,P40925,P41222,P43121,P43251,P43652,P49588,P49908,P51884,P54289,P55290,P60174,P61278,P61626,P61769,P61916,P80748,P98160,Q02818,Q06481,Q08380,Q12805,Q12841,Q12907,Q13283,Q13332,Q13449,Q13451,Q13740,Q14118,Q14508,Q14515,Q14624,Q15904,Q16270,Q16610,Q562R1,Q6UX71,Q6UXB8,Q6UXD5,Q7Z3B1,Q7Z5P9,Q8IWV7,Q8N2S1,Q8NBJ4,Q8NE71,Q92520,Q92823,Q92876,Q96BZ4,Q96KN2,Q96PD5,Q96S96,Q99435,Q99674,Q99683,Q99829,Q99832,Q99969,Q9BY67,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7,pred_month_plus_0,pred_month_plus_6,updrs_1_plus_6,updrs_2_plus_6,updrs_3_plus_6,updrs_4_plus_6,pred_month_plus_12,updrs_1_plus_12,updrs_2_plus_12,updrs_3_plus_12,updrs_4_plus_12,pred_month_plus_24,updrs_1_plus_24,updrs_2_plus_24,updrs_3_plus_24,updrs_4_plus_24
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,11254.3,732430.0,39585.8,41526.9,31238.0,4202.71,177775.0,62898.2,333376.0,166850.0,98746.3,6380.35,67567.6,64117.8,1181230.0,688909.0,109541.0,3956470.0,111619.0,347865.0,71835.4,637630.0,2676370.0,14415900.0,2025890.0,1984650.0,1953020.0,3916980.0,13033.3,117641.0,43235500.0,1221530.0,136544.0,6978.98,NaN,11386.00,126829.0,110750.0,9600.69,4950980.0,2121640.0,3521370.0,2224420.0,714486.0,18430.1,1829650.0,39221.5,87890.4,11270600.0,15487900.0,842865.0,24917.0,88723.3,668070.0,568133.0,209127.0,161430.0,162768.0,1613890.0,370958.0,2541830.0,571002.0,396955.0,5584050.0,2625460.0,23735400.0,390239000.0,3857700.0,32075400.0,4774060.0,NaN,1156340.0,34250.6,507483.0,79350.6,208598.0,61824.4,37354.5,186082.0,1510720.0,458306.0,17977.8,603685.0,210539.0,344149.0,9137820.0,1029290.0,108603.0,8667.28,578350.0,108005.0,9621.47,2607410.0,31886.0,100481.0,831886.0,12879.8,172731.0,54219.0,334859.0,1401860.0,29748.3,49802.5,1324370.0,23399.4,62755.1,22865.0,860978.0,125774.0,530914.0,1589560.0,26873.4,322370.0,9988.60,215991.0,430074.0,399489.0,140548.0,631794.0,11575700.0,84143.3,106498.0,104399.0,132270.0,50588.8,237956.0,33138.7,8755.34,572295.0,36107.4,44112.6,92071.9,91637.0,302429.0,47171.0,325214.0,36079.3,9451.71,438323.0,10886.2,1106490.0,112070.0,36232.9,528485.0,36406.9,27236.0,1673460.0,862077.0,922153.0,13998.0,89747.3,156923.0,NaN,56648.9,502305.0,2736190.0,12466.0,74564.3,31201.3,33921600.0,64855.6,27257.2,410727.0,15043.9,23457.2,380950.0,65910.6,43209.0,NaN,6323.72,23749.4,701219.0,449831.0,32505.8,21970.1,25613.8,39504.7,268657.0,1611130.0,39418.2,50763.0,663339.0,22053.5,NaN,3170450.0,34672.4,266247.0,9649.82,512157.0,161173.0,46305.4,1451950.0,221526.0,31592.7,18041.0,59467.7,31029.1,116200.0,519708.0,57246.2,10802.8,29027.7,36202.0,982965.0,200896.0,1282260.0,13189.3,633503.0,123757.0,34868.9,3417.71,34550.6,NaN,NaN,64488.0,22021.6,35217.8,365475.0,35528.0,97005.6,23122.5,60912.6,408698.0,NaN,29758.8,23833.7,18953.5,0,6.0,8.0,10.0,34.0,NaN,12.

In [9]:
def calculate_month_trend_predicitons(pred_month, trend):
    if target == 'updrs_4': 
        pred_month = pred_month.clip(54, None)
    return trend[0] + pred_month * trend[1]

# original
target_to_trend = {
    'updrs_1': [5.394793062665313, 0.027091086167821344],
    'updrs_2': [5.469498130092747, 0.02824188329658148],
    'updrs_3': [21.182145576879183, 0.08897763331790556],
    'updrs_4': [-4.434453480103724, 0.07531448585334258]
}

# opt_linear
target_to_trend_opt = {
    'updrs_1': [5.069663560845211, 0.03480012196679531],
    'updrs_2': [4.932927377561229, 0.04267552599462405],
    'updrs_3': [19.762037079411606, 0.1309360847988586],
    'updrs_4': [-5.812643019993671, 0.1010114148558041]
}
    

# opt_multi
target_to_trend_multi = {
    'updrs_1': [4.507702474332501,  0.10062226891281667,  -0.0007645063347058566,  -0.09959957526314371],
    'updrs_2': [5.4091224778513025, -0.0024915739431735154,  0.0006672555419938409,  0.07037380527048327],
    'updrs_3': [21.4174950327159,  -0.04435463134786311,  0.002270401674859704,  0.15815142637299842],
    'updrs_4': [-5.188554809911633,  0.0970618495564384,  2.4753489351227137e-11,  -0.09390397631718975]
}

In [10]:
def calculate_predicitons_protein(protein, pred_month, protein_shift):
    trend_pred_month = target_to_trend[target]
    pred_month_trend = calculate_month_trend_predicitons(pred_month=pred_month, trend=trend_pred_month)
    return np.round(pred_month_trend + protein_shift)

def function_to_minimize(x):
    metric = smape_plus_1(
        y_true=y_true_array, 
        y_pred=calculate_predicitons_protein(
            protein=protein_array,
            pred_month=pred_month_array,
            protein_shift=x[0]
        )
    )
    return metric

In [11]:
def find_best_const(train_clinical_all_filtered, target):
    columns_with_target = [f'{target}_plus_{plus_month}' for plus_month in [0, 6, 12, 24]]
    columns_with_pred_month = [f'pred_month_plus_{plus_month}' for plus_month in [0, 6, 12, 24]]
    global y_true_array
    global pred_month_array
    global protein_array
    y_true_array = train_clinical_all_filtered[columns_with_target].values.ravel()
    pred_month_array = train_clinical_all_filtered[columns_with_pred_month].values.ravel()
    protein_array = np.concatenate([train_clinical_all_filtered[feature].values] * 4)
    result = minimize(
        fun=function_to_minimize,
        x0=[0.0],
        method='Powell'
    ).x[0]
    return result

## Plot shifts

In [12]:
feature = 'O15240'
quantiles = [0, 0.05, 0.95, 1.0]

df_plot = []
for quantile_low, quantile_high in tqdm(zip(quantiles[:-1], quantiles[1:])):
    item = {
        'quantile_low': quantile_low,
        'quantile_high': quantile_high,
        'quantile_middle': (quantile_low + quantile_high) / 2
    }
    quantile_low_value = train_clinical_all[feature].quantile(quantile_low)
    quantile_high_value = train_clinical_all[feature].quantile(quantile_high)
    item['quantile_low_value'] = quantile_low_value
    item['quantile_high_value'] = quantile_high_value
    
    if quantile_high == 1:
        quantile_high_value += 0.00001
        
    train_clinical_all_filtered = train_clinical_all[
        (train_clinical_all[feature] >= quantile_low_value)
        & (train_clinical_all[feature] < quantile_high_value)
    ]
    for target in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
        item[f'{target}_shift'] = find_best_const(train_clinical_all_filtered, target)
    df_plot.append(item)
    
df_plot = pd.DataFrame(df_plot)

0it [00:00, ?it/s]

In [13]:
for target in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
    fig = px.line(
        df_plot,
        y=f'{target}_shift',
        x='quantile_middle',
        title=feature + ' ' + target
    )
    fig.show()

## Find shifts

In [14]:
npx_groups = [
    {'quantile_low': 0.0, 'quantile_high': 0.05},
    {'quantile_low': 0.05, 'quantile_high': 0.95},
    {'quantile_low': 0.95, 'quantile_high': 1.0},
]
target_to_npx_groups_shift = defaultdict(list)

for target in ['updrs_1', 'updrs_2', 'updrs_3']:
    for npx_group in npx_groups:
        item = npx_group.copy()
        item['feature'] = feature
        
        if item['quantile_low'] == 0:
            item['quantile_low_value'] = -np.inf
        else:
            item['quantile_low_value'] = train_clinical_all[feature].quantile(item['quantile_low'])
            
        if item['quantile_high'] == 1:
            item['quantile_high_value'] = np.inf
        else: 
            item['quantile_high_value'] = train_clinical_all[feature].quantile(item['quantile_high'])

        train_clinical_all_filtered = train_clinical_all[
            (train_clinical_all[feature] >= item['quantile_low_value'])
            & (train_clinical_all[feature] < item['quantile_high_value'])
        ]
        
        item['shift'] = find_best_const(train_clinical_all_filtered, target)
        target_to_npx_groups_shift[target].append(item)

## Predictions

In [15]:
models = {
    '1':[],
    '2':[],
    '3':[],
    '4':[]
}

for u in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
    u_num = u.split('_')[1]
    for ii in range(1, 6):
        filename = f'{model_path}/{u}_fold{ii}.pkl'      
        print(filename)
        with open(filename, mode='rb') as f: 
            model = pickle.load(f)       

        models[u_num].append(model)        

/kaggle/input/lgbmmodels/lgbm_models/updrs_1_fold1.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_1_fold2.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_1_fold3.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_1_fold4.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_1_fold5.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_2_fold1.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_2_fold2.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_2_fold3.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_2_fold4.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_2_fold5.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_3_fold1.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_3_fold2.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_3_fold3.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_3_fold4.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_3_fold5.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_4_fold1.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_4_fold2.pkl
/kaggle/input/lgbmmodels/lgbm_models/updrs_4_fold3.pkl
/kaggle/in

In [16]:
amp_pd_peptide.make_env.func_dict['__called__'] = False
env = amp_pd_peptide.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test files

proteins_features_all = pd.DataFrame()
# The API will deliver four dataframes in this specific order:
for test_clinical_data, test_peptides, test_proteins, sample_submission in iter_test:
    sample_submission['patient_id'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[0]))
    sample_submission['visit_month'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[1]))
    sample_submission['target_name'] = sample_submission['prediction_id'].map(lambda x: 'updrs_' + x.split('_')[3])
    sample_submission['plus_month'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[5]))
    sample_submission['pred_month'] = sample_submission['visit_month'] + sample_submission['plus_month']
    sample_submission['visit_id'] = sample_submission['patient_id'].astype(str) + '_' + sample_submission['visit_month'].astype(str)
    
    try:
    
        proteins_features = pd.pivot_table(test_proteins, values='NPX', index='visit_id', columns='UniProt', aggfunc='sum')
        proteins_features['visit_id'] = proteins_features.index
        proteins_features_all = pd.concat([proteins_features_all, proteins_features])
        proteins_features_all['patient_id'] = proteins_features_all.index.map(lambda x: int(x.split('_')[0]))
        proteins_features_all[proteins_features.columns] = proteins_features_all.groupby('patient_id')[proteins_features.columns].\
                                                                                                       fillna(method='ffill')
        proteins_features = proteins_features_all.groupby('patient_id', as_index=False).last()

        sample_submission = sample_submission.merge(
            proteins_features,
            on='patient_id',
            how='left'
        )
        
    except:
        pass

    for i in range(1, 5):
        target = f'updrs_{i}'
        mask_target = sample_submission['target_name'] == target
        sample_submission.loc[mask_target, 'rating'] = calculate_month_trend_predicitons(
            pred_month=sample_submission.loc[mask_target, 'pred_month'],
            trend=target_to_trend[target]
        )
        
        
        try:
            for item in target_to_npx_groups_shift[target]:
                feature = item['feature']
                mask_feature_range = mask_target & (
                    (sample_submission[feature] >= item['quantile_low_value'])
                    & (sample_submission[feature] < item['quantile_high_value'])
                )
                sample_submission.loc[mask_feature_range, 'rating'] += item['shift']
        except:
            pass
        
        sample_submission.loc[mask_target, 'rating'] = np.round(sample_submission.loc[mask_target, 'rating'])
    
    
    try: 
        
        ensemble_submission1 = predict_lgbm.prediction(
            CFG, 
            test_clinical_data, 
            test_peptides, 
            test_proteins, 
            sample_submission, 
            models,
            ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4'],
            target_to_trend_opt
        )

        ensemble_submission2 = predict_lgbm.prediction(
            CFG, 
            test_clinical_data, 
            test_peptides, 
            test_proteins, 
            sample_submission, 
            models,
            ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4'],
            target_to_trend_multi
        )

        final_submission = pd.merge(
            sample_submission[['prediction_id', 'rating']].rename(columns = {'rating':'rating0'}),
            ensemble_submission1[['prediction_id', 'rating']].rename(columns = {'rating':'rating1'}),
            on = 'prediction_id',
            how = 'left'
        )

        final_submission = pd.merge(
            final_submission,
            ensemble_submission2[['prediction_id', 'rating']].rename(columns = {'rating':'rating2'}),
            on = 'prediction_id',
            how = 'left'
        )

        final_submission['rating'] = (final_submission['rating0']*0.6) + (final_submission['rating1']*0.2) + (final_submission['rating2']*0.2)

        # call the env.predict for every iteration
        env.predict(final_submission[['prediction_id', 'rating']])
        
    except:
        env.predict(sample_submission[['prediction_id', 'rating']])

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
clinical_read finished
  visit_id  patient_id  visit_month  updrs_1  updrs_2  updrs_3  updrs_4  \
0     55_0          55            0     10.0      6.0     15.0      NaN   
1     55_3          55            3     10.0      7.0     25.0      NaN   
2     55_6          55            6      8.0     10.0     34.0      NaN   
3     55_9          55            9      8.0      9.0     30.0      0.0   
4    55_12          55           12     10.0     10.0     41.0      0.0   

   upd23b_clinical_state_on_medication  
0                                   -1  
1                                   -1  
2                                   -1  
3                                    0  
4                                    0  


protein_read finished
  visit_id  visit_month  patient_id UniProt       NPX
0     55_0            0          55  O00391   11254.3
1     55_0            0